In [22]:
## 업로드 시 삭제하기
client_id = "0Em_XGR51iU6sqnSE9EB"
client_secret = "Qn4EbSzOq2"

# 광고 api 키값
BASE_URL = 'https://api.naver.com'
API_KEY = '010000000018b29124e84eec4b325f8ab0a2149148244fcbc4c613ea3ddc7a5869e3066f0b'
SECRET_KEY = 'AQAAAAAYspEk6E7sSzJfirCiFJFIoiQJ10rdN57vR/xoMRKsqQ=='
CUSTOMER_ID = 3116501

# 네이버 검색 > 쇼핑API
<pr>
    : 네이버 쇼핑 페이지에서 cat_keyword를 검색한 후 상위 페이지의 데이터만 긁어옴
</pr>

In [3]:
import pandas as pd
import os
import sys
import urllib.request
import json
import numpy as np

cat_keyword = '냉장고'
encText = urllib.parse.quote(cat_keyword)
disText = urllib.parse.quote("80") # 상위 nn개의 상품 수집
filter = urllib.parse.quote("naverpay")
exc = urllib.parse.quote("cbshop:rental:used")

url = "https://openapi.naver.com/v1/search/shop.json?query=" + encText + '&display=' + disText# JSON 결과

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print('Fin')
else:
    print("Error Code:" + rescode)

Fin


In [17]:
# 총 상품 수
total_pd = json.loads(response_body.decode('utf-8'))['total']
print(total_pd)

5218134


In [6]:
# 결과 json 값을 df로 변환

df = pd.DataFrame(json.loads(response_body.decode('utf-8'))['items'])
df.head()

,title,link,image,lprice,hprice,mallName,productId,productType,brand,maker,category1,category2,category3,category4
0,LG전자 오브제컬렉션 4도어 870L 네이처 베이지 (T873MEE312),https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_333653...,1499000,,네이버,33365337618,1,오브제컬렉션,LG전자,디지털/가전,주방가전,냉장고,양문형냉장고
1,삼성전자 삼성 2도어 306L 엘레강트블랙 (RB30R4051B1),https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_208583...,439190,,네이버,20858302566,1,삼성,삼성전자,디지털/가전,주방가전,냉장고,일반형냉장고
2,LG전자 오브제컬렉션 4도어 870L 네이처 화이트 (T873MWW111),https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_333664...,1549000,,네이버,33366468618,1,오브제컬렉션,LG전자,디지털/가전,주방가전,냉장고,양문형냉장고
3,삼성전자 비스포크 4도어 875L 매트 멜로우 화이트 (RF84C906B4W),https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_387230...,1308450,,네이버,38723041618,1,비스포크,삼성전자,디지털/가전,주방가전,냉장고,양문형냉장고
4,삼성전자 삼성 2도어 846L 스노우 화이트 (RS84B5041WW),https://search.shopping.naver.com/gate.nhn?id=...,https://shopping-phinf.pstatic.net/main_347938...,969000,,네이버,34793800618,1,삼성,삼성전자,디지털/가전,주방가전,냉장고,양문형냉장고


In [7]:
# 상위 카테고리 분포 
df['cat_merge'] = df['category1'] + '-' + df['category2'] + '-' + df['category3'] + '-' + df['category4'] 
mdd = df['cat_merge'].value_counts().reset_index()
mdd['per'] = mdd['count'].apply(lambda x : np.round(x/80,2)*100) # 카테고리별 비율
mdd.head()

,cat_merge,count,per
0,디지털/가전-주방가전-냉장고-양문형냉장고,49,61.0
1,디지털/가전-주방가전-냉장고-일반형냉장고,25,31.0
2,디지털/가전-주방가전-전용냉장고-와인냉장고,3,4.0
3,디지털/가전-주방가전-전용냉장고-제빙기,2,2.0
4,디지털/가전-주방가전-전용냉장고-쇼케이스,1,1.0


# 네이버 광고 API
<pr>
    : keyword와 관련된 연관검색어 및 해당 검색어들의 검색량, 클릭수에 대한 데이터 제공
</pr>

In [9]:
import hashlib, hmac , base64

import time
import random
import requests
import pandas as pd
import signaturehelper
import json
import numpy as np


def run_naver_ad_api(hint_keyword) :
    CUSTOMER_ID = 2884380
    API_KEY = '0100000000b19b37b5629cb4547ae39519e376a8976a28096d0974036d2763d60500af7c08'
    SECRET_KEY = 'AQAAAACxmze1Ypy0VHrjlRnjdqiX8/QfwgTEda5egGzpPYQI8A=='

    def get_header(method, uri, api_key, secret_key, customer_id):
        timestamp = str(round(time.time() * 1000))
        signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
        return {'Content-Type': 'application/json; charset=UTF-8', 'X-Timestamp': timestamp, 'X-API-KEY': API_KEY, 'X-Customer': str(CUSTOMER_ID), 'X-Signature': signature}


    uri = '/keywordstool'
    method = 'GET'

    query = {
        'biztpId' : '',
        'hintKeywords' : hint_keyword,
        'event' : '',
        'month' : 1,
        'showDetail' : '1'}
    
    BASE_URL = 'https://api.searchad.naver.com'

    r = requests.get(
    BASE_URL  + uri,
    params = query,
    headers = get_header(method,uri,API_KEY,SECRET_KEY,CUSTOMER_ID))

    r_data = r.json()
    # print(r_data.keys())
    print('연관된 키워드 수 : ', len(r_data['keywordList']))
    # print(r_data['keywordList'][0])

    r_data = pd.DataFrame(r_data['keywordList'])
    # r_data.columns = ['연관키워드','월간검색수(PC)','월간검색수(모바일)','월평균클릭수(PC)','월평균클릭수(모바일)','월평균클릭률(PC)','월평균클릭률(모바일)','월평균노출광고수','경쟁정도']
    r_data['monthlyPcQcCnt'] = r_data['monthlyPcQcCnt'].apply(lambda x : int(str(x).replace('<','')))
    r_data['monthlyMobileQcCnt'] = r_data['monthlyMobileQcCnt'].apply(lambda x : int(str(x).replace('<','')))
    r_data['total']=r_data['monthlyMobileQcCnt']+r_data['monthlyPcQcCnt']
    r_data['totla_click']=r_data['monthlyAvePcClkCnt']+r_data['monthlyAveMobileClkCnt']
    return r_data


In [11]:
r_data.head()

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx,total,totla_click
0,냉장고,44700,173800,287.5,2788.6,0.67,1.73,10,중간,218500,3076.1
1,에어프라이어,32800,239700,246.1,4421.8,0.77,1.99,10,중간,272500,4667.9
2,중고가구,2630,10400,105.4,483.6,4.14,4.97,10,중간,13030,589.0
3,전자레인지선반,2740,16400,12.1,186.4,0.46,1.22,10,중간,19140,198.5
4,세탁기,18100,102200,144.9,1576.4,0.83,1.66,10,중간,120300,1721.3


In [10]:
# 연관키워드 및 검색수 
cat_keyword = '냉장고'
r_data = run_naver_ad_api(cat_keyword)

r_data[['relKeyword','total']].head()

연관된 키워드 수 :  1200


,relKeyword,total
0,냉장고,218500
1,에어프라이어,272500
2,중고가구,13030
3,전자레인지선반,19140
4,세탁기,120300


In [12]:
# 월평균 클릭수

print('전체 :',r_data['monthlyAvePcClkCnt'][0]+r_data['monthlyAveMobileClkCnt'][0])
print('모바일 :',r_data['monthlyAveMobileClkCnt'][0])
print('PC :',r_data['monthlyAvePcClkCnt'][0])

전체 : 3076.1
모바일 : 2788.6
PC : 287.5


In [16]:
# 월평균 클릭율 : 검색수 / 전체검색수 + 월평균클릭수

print('전체 :',
      round(r_data['monthlyAveMobileClkCnt'][0]/(r_data['monthlyAvePcClkCnt'][0]+r_data['monthlyAveMobileClkCnt'][0])*r_data['monthlyAveMobileCtr'][0] + 
      r_data['monthlyAvePcClkCnt'][0]/(r_data['monthlyAvePcClkCnt'][0]+r_data['monthlyAveMobileClkCnt'][0]*r_data['monthlyAvePcCtr'][0]),2)
      )

print('모바일 :',r_data['monthlyAveMobileCtr'][0])

print('PC :',r_data['monthlyAvePcCtr'][0])

전체 : 1.7
모바일 : 1.73
PC : 0.67


In [20]:
#월평균클릭경쟁률 = 총상품수/총클릭수
print('월평균클릭경쟁률 :', total_pd/r_data['total'][0])

#  월 평균 노출 광고수
print('월평균노출광고수 :',r_data['plAvgDepth'][0])

# 총 검색량
print('총검색량 :', r_data['total'][0])

# 월간 검색량
print('월간 검색량 :', r_data['monthlyPcQcCnt'][0]+r_data['monthlyMobileQcCnt'][0])

월평균클릭경쟁률 : 23.88162013729977
월평균노출광고수 : 10
총검색량 : 218500
월간 검색량 : 218500


In [21]:
r_data

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx,total,totla_click
0,냉장고,44700,173800,287.5,2788.6,0.67,1.73,10,중간,218500,3076.1
1,에어프라이어,32800,239700,246.1,4421.8,0.77,1.99,10,중간,272500,4667.9
2,중고가구,2630,10400,105.4,483.6,4.14,4.97,10,중간,13030,589.0
3,전자레인지선반,2740,16400,12.1,186.4,0.46,1.22,10,중간,19140,198.5
4,세탁기,18100,102200,144.9,1576.4,0.83,1.66,10,중간,120300,1721.3
...,...,...,...,...,...,...,...,...,...,...,...
1195,보드장갑,1940,9140,15.0,169.8,0.80,1.98,15,높음,11080,184.8
1196,중학교가방,450,5270,3.9,75.3,0.91,1.52,7,중간,5720,79.2
1197,웰리힐리스키장,4720,28500,25.7,313.6,0.57,1.19,15,높음,33220,339.3
1198,바디오일추천,540,5490,1.5,48.5,0.28,0.96,7,중간,6030,50.0


# 네이버 데이터랩 API > 검색클릭추이
<pr>
    : 네이버 쇼핑에서 특정 쇼핑 분야의 검색클릭추이를 사용자의 특징(기기, 성별, 연령 등)별로 조회한 데이터를 JSON 형식으로 반환
</pr>

In [23]:
import os
import sys
import urllib.request


def naver_keyword_click_de_age_gen(api_type,startDate,endDate, timeUnit, category,keyword, device = '', gender = '', ages = []) :
    # client_id = ""
    # client_secret = ""

    # 연령
    if api_type == 'age' :
        url = "https://openapi.naver.com/v1/datalab/shopping/category/keyword/age"
        body = {
            "startDate" : startDate,
            "endDate" : endDate,
            "timeUnit" : timeUnit,
            "category" : category,
            'keyword' : keyword,
            "device" : device,
            "gender" : gender,
            "ages" : ages
        }

    # 기기
    elif api_type == 'device' :
        url = "https://openapi.naver.com/v1/datalab/shopping/category/keyword/device"
        body = {
            "startDate" : startDate,
            "endDate" : endDate,
            "timeUnit" : timeUnit,
            "category" : category,
            'keyword' : keyword,
            "device" : device,
            "gender" : gender,
            "ages" : ages
        }

    # 성별
    else :
        url = "https://openapi.naver.com/v1/datalab/shopping/category/keyword/gender"
        body = {
            "startDate" : startDate,
            "endDate" : endDate,
            "timeUnit" : timeUnit,
            "category" : category,
            'keyword' : keyword,
            "device" : device,
            "gender" : gender,
            "ages" : ages
        }

    body = json.dumps(body)


    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    request.add_header("Content-Type","application/json")
    response = urllib.request.urlopen(request, data=body.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        response_body = json.loads(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)
    

    response_results = pd.DataFrame()
    for data in response_body['results'] :
        result = pd.DataFrame(data['data'])
        result['title'] = data['title']
        response_results = pd.concat([response_results,result])


    return response_body,response_results

## 성별 검색 비율

In [24]:
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from datetime import datetime

toda = datetime.now() #현재시간

# 해당 부분에서 기간별 값을 조정 가능
''' 
one_week_ago = toda - timedelta(weeks=1)
one_week_ago = one_week_ago.strftime('%Y-%m-%d')
one_week_ago= str(one_week_ago)
'''
time_month = toda - relativedelta(months=1) # 1달 이전 날짜
time_month = time_month.strftime('%Y-%m-%d')
time_month= str(time_month)



today = str(datetime.now().date()) # 오늘
print(today,time_month)

startDate = time_month
endDate = today
timeUnit = 'week'
category = '50001724'
# 선풍기 : 50001420
# 냉장고 : 50001430
keyword = '와플제조기'
body, response_body = naver_keyword_click_de_age_gen('gender',startDate,endDate,timeUnit,category,keyword)

2024-01-29 2023-12-29


In [25]:
gen_df=response_body
gen_df

,period,ratio,group,title
0,2023-12-25,100.00000,f,와플제조기
1,2023-12-25,13.33333,m,와플제조기
2,2024-01-01,100.00000,f,와플제조기
3,2024-01-01,40.00000,m,와플제조기
4,2024-01-08,68.33333,f,와플제조기
5,2024-01-08,13.33333,m,와플제조기
6,2024-01-15,91.66666,f,와플제조기
7,2024-01-15,23.33333,m,와플제조기
8,2024-01-22,98.33333,f,와플제조기
9,2024-01-22,25.00000,m,와플제조기


In [30]:
# 성별 클릭 비율
gen_total = gen_df['ratio'].sum()
gb1 = gen_df.groupby(['group'])['ratio'].sum().reset_index()
gb1['PER'] = gb1['ratio'].apply(lambda x : np.round((x/gen_total),3)*100)
gb1

,group,ratio,PER
0,f,458.33332,79.9
1,m,114.99999,20.1


## 연령별 검색 비율

In [26]:
###연령별
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from datetime import datetime

toda = datetime.now() #현재시간

# 해당부분에서 주간/월간 조절 가능
time_month = toda - relativedelta(months=1) # 1달 이전 날짜
time_month = time_month.strftime('%Y-%m-%d')
time_month= str(time_month)

today = str(datetime.now().date()) # 오늘
print(today,time_month)

startDate = time_month
endDate = today
timeUnit = 'week'
category = '50001430'
# 선풍기 : 50001420
# 냉장고 : 50001430
keyword = '냉장고'
response_body = naver_keyword_click_de_age_gen('age',startDate,endDate,timeUnit,category,keyword)

2024-01-29 2023-12-29


In [27]:
age_df=response_body[1]
age_df

,period,ratio,group,title
0,2023-12-25,0.59684,10,냉장고
1,2023-12-25,19.41042,20,냉장고
2,2023-12-25,64.77060,30,냉장고
3,2023-12-25,83.41291,40,냉장고
4,2023-12-25,45.07992,50,냉장고
5,2023-12-25,12.26385,60,냉장고
6,2024-01-01,0.48266,10,냉장고
7,2024-01-01,17.94166,20,냉장고
8,2024-01-01,73.67656,30,냉장고
9,2024-01-01,99.75607,40,냉장고


In [31]:
# 연령별 클릭비율
age_total = age_df['ratio'].sum()
gb2 = age_df.groupby(['group'])['ratio'].sum().reset_index()
gb2['PER'] = gb2['ratio'].apply(lambda x : np.round((x/age_total),3)*100)
gb2

,group,ratio,PER
0,10,3.03091,0.2
1,20,92.14239,7.3
2,30,356.26944,28.4
3,40,464.92110,37.0
4,50,263.86234,21.0
5,60,75.89265,6.0


## 기기별 검색 비율

In [28]:
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from datetime import datetime

toda = datetime.now() #현재시간
# 해당부분에서 주간/월간 조절 가능
time_month = toda - relativedelta(months=1) # 1달 이전 날짜
time_month = time_month.strftime('%Y-%m-%d')
time_month= str(time_month)

today = str(datetime.now().date()) # 오늘
print(today,time_month)

startDate = time_month
endDate = today
timeUnit = 'week' # month, week, date에 따라서 기간별 값을 조절 가능
category = '50001408'
# 선풍기 : 50001420
# 냉장고 : 50001430
# 세탁기 : 50001408
keyword = '세탁기'
response_body = naver_keyword_click_de_age_gen('device',startDate,endDate,timeUnit,category,keyword)

2024-01-29 2023-12-29


In [29]:
device_df=response_body[1]
device_df

,period,ratio,group,title
0,2023-12-25,82.95061,mo,세탁기
1,2023-12-25,16.98173,pc,세탁기
2,2024-01-01,100.00000,mo,세탁기
3,2024-01-01,19.25418,pc,세탁기
4,2024-01-08,91.17284,mo,세탁기
5,2024-01-08,18.70895,pc,세탁기
6,2024-01-15,82.64814,mo,세탁기
7,2024-01-15,18.53384,pc,세탁기
8,2024-01-22,78.52907,mo,세탁기
9,2024-01-22,17.85728,pc,세탁기


In [32]:
# 기기별 클릭 비율
device_total = device_df['ratio'].sum()
gb3 = device_df.groupby(['group'])['ratio'].sum().reset_index()
gb3['PER'] = gb3['ratio'].apply(lambda x : np.round((x/device_total),3)*100)
gb3

,group,ratio,PER
0,mo,435.30066,82.7
1,pc,91.33598,17.3


# 네이버 데이터랩 API > 검색추이 데이터
<pr> 
    : 네이버 통합검색에서 키워드에 대한 검색 추이 데이터를 반환
        * 기간 내 최대 검색량을 100으로 둔 상대적 검색량값  반환
</pr>

In [34]:
import os
import sys
import urllib.request
import json
import pandas as pd
import matplotlib.pyplot as plt

device = 'mo'
cat_keyword = '냉장고'

url = "https://openapi.naver.com/v1/datalab/search" # 네이버 데이터랩 통합 검색어 트렌드 api 실행 (한달전부터 오늘까지의, 키워드 검색수 비율 도출)
body = "{\"startDate\":\""+time_month+"\",\"endDate\":\""+today+"\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\""+cat_keyword+"\",\"keywords\":[\""+cat_keyword+"\"]}],\"device\":\""+device+"\"}";
requested = urllib.request.Request(url)
requested.add_header("X-Naver-Client-Id", client_id)
requested.add_header("X-Naver-Client-Secret", client_secret)
requested.add_header("Content-Type", "application/json")
response = urllib.request.urlopen(requested, data=body.encode("utf-8"))
rescode = response.getcode()

if(rescode==200):
    response_body = response.read()
    output_data = response_body.decode('utf-8')
else:
    print('Error code:'+ rescode)
    pass

result = json.loads(output_data)

In [35]:
search_data=pd.DataFrame(result['results'][0]['data'])
search_data['keyword'] = cat_keyword
search_data.head() # 냉장고 검색비율(1년치-일단위)

,period,ratio,keyword
0,2023-12-29,68.52754,냉장고
1,2023-12-30,80.49032,냉장고
2,2023-12-31,80.53463,냉장고
3,2024-01-01,88.68704,냉장고
4,2024-01-02,83.73947,냉장고


In [37]:
# 요일별 검색량 추론

from dateutil.parser import parse
    # datetime.datetime.strptime() 클래스 메서드 : 문자열에 맞는 형식 문자열을 사용자가 제공해야함
        # VS dateutil.parser의 parse 명령 : 자동으로 형식 문자열을 찾아 datetime.datetime 클래스 객체를 만듦

days = ['월','화', '수', '목', '금', '토', '일']

search_data['Date'] = search_data['period'].apply(lambda x : parse(x))
search_data['weekday'] = search_data['Date'].apply(lambda x : days[x.weekday()])
                                                        # x의 날짜를 나타내는 숫자를 반환
search_data['Y'] = search_data['period'].apply(lambda x : x.split('-')[0])
search_data['M'] = search_data['period'].apply(lambda x : x.split('-')[1])
search_data = search_data[['Date','Y','M','weekday','ratio','keyword']]
search_data.head()

,Date,Y,M,weekday,ratio,keyword
0,2023-12-29,2023,12,금,68.52754,냉장고
1,2023-12-30,2023,12,토,80.49032,냉장고
2,2023-12-31,2023,12,일,80.53463,냉장고
3,2024-01-01,2024,01,월,88.68704,냉장고
4,2024-01-02,2024,01,화,83.73947,냉장고


In [38]:
# 요일별 검색 비율
day_search = search_data.groupby(['weekday'])['ratio'].sum().reset_index()
total = day_search['ratio'].sum()
day_search['PER'] = day_search['ratio'].apply(lambda x : np.round(x/total,3)*100)
day_search

,weekday,ratio,PER
0,금,365.57374,14.3
1,목,307.85701,12.0
2,수,314.96085,12.3
3,월,342.94784,13.4
4,일,466.06112,18.2
5,토,436.09508,17.0
6,화,328.22328,12.8


In [39]:
# 월별 검색비율
month_search = search_data.groupby(['Y','M'])['ratio'].sum().reset_index()
total = month_search['ratio'].sum()
month_search['PER'] = month_search['ratio'].apply(lambda x : np.round(x/total,3)*100)

month_search.head()

,Y,M,ratio,PER
0,2023,12,229.55249,9.0
1,2024,01,2332.16643,91.0


# 네이버 검색 API

## 뉴스 검색 결과

In [40]:
def getresult(client_id,client_secret,query,display=100,start=1,sort='sim'):
    encText = urllib.parse.quote(query)
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + \
    "&display=" + str(display) + "&start=" + str(start) + "&sort=" + sort

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        response_json = json.loads(response_body)
    else:
        print("Error Code:" + rescode)
    print(f"총 상품수 : {response_json['total']}")
    return pd.DataFrame(response_json['items']),response_json['total']

In [41]:
query = '세탁기'
display=100
start=1
sort='date'

news_result,news_total= getresult(client_id,client_secret,query,display,start,sort)

        # news_total : 총 상품수 카운트
        # result : 결과 데이터프레임
news_result.head()

총 상품수 : 317944


,title,originallink,link,description,pubDate
0,"LH, 올해 자립준비청년 매입임대주택 상시 접수 시작",http://www.seoulwire.com/news/articleView.html...,http://www.seoulwire.com/news/articleView.html...,"공급되는 주택은 청년 등 입주민 수요를 반영해 냉장고, <b>세탁기</b>, 에어컨...","Mon, 29 Jan 2024 16:44:00 +0900"
1,"LH, 올해 자립준비청년 매입임대주택 상시 접수 시작",http://www.newscammp.co.kr/article/view/ncp202...,http://www.newscammp.co.kr/article/view/ncp202...,"공급되는 주택은 청년 등 입주민 수요를 반영해 냉장고, <b>세탁기</b>, 에어컨...","Mon, 29 Jan 2024 16:34:00 +0900"
2,"LH, 자립준비청년 매입임대주택 상시 접수 시작",http://theviewers.co.kr/View.aspx?No=3109654,http://theviewers.co.kr/View.aspx?No=3109654,"공급되는 주택은 청년 등 입주민 수요를 반영해 냉장고, <b>세탁기</b>, 에어컨...","Mon, 29 Jan 2024 16:32:00 +0900"
3,"LH, 자립준비청년 매입임대주택 400호 공급…최장 10년 거주 가능",http://www.g-enews.com/ko-kr/news/article/news...,http://www.g-enews.com/ko-kr/news/article/news...,"냉장고, <b>세탁기</b>, 에어컨 등 필수 가전제품이 갖춰져 있으며, 최장 10...","Mon, 29 Jan 2024 16:30:00 +0900"
4,"&quot;자립준비청년들의 든든한 주거 버팀목 될 것&quot;…LH, 자립준비청년...",http://www.thefirstmedia.net/news/articleView....,http://www.thefirstmedia.net/news/articleView....,"서울 매입임대주택 모습(사진: LH) 공급되는 주택은 냉장고, <b>세탁기</b>,...","Mon, 29 Jan 2024 16:24:00 +0900"


## 블로그 검색 결과

In [42]:
def getresult(client_id,client_secret,query,display=100,start=1,sort='sim'):
    encText = urllib.parse.quote(query)
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText + \
    "&display=" + str(display) + "&start=" + str(start) + "&sort=" + sort

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        response_json = json.loads(response_body)
    else:
        print("Error Code:" + rescode)
    print(f"게시글 수 : {response_json['total']}")
    return pd.DataFrame(response_json['items']),response_json['total']

In [43]:
query = '냉장고'
display=100
start=1
sort='sim'

blog_result,blog_total= getresult(client_id,client_secret,query,display,start,sort)

blog_result['title'] = blog_result['title'].apply(lambda x: x.replace('&quot;','').replace('&apos;','').replace('<b>','').replace('</b>','').strip())
blog_result['description'] = blog_result['description'].apply(lambda x: x.replace('<b>','').replace('</b>','').replace('&quot;','').replace('&apos;','').strip())

blog_result.head(5)

게시글 수 : 13002827


,title,link,description,bloggername,bloggerlink,postdate
0,젠우드 빌트인 틈새장 냉장고장 빈틈 수납장 내돈내산 후기,https://blog.naver.com/joojoo5623/223322607913,이사 온 지 약 2개월가량 되었을 무렵 문뜩 눈에 거슬리기 시작한 냉장고장..! 이...,바삭바삭한 일상그리기❣,blog.naver.com/joojoo5623,20240115
1,"비스포크키친핏 냉장고장, 싱크대교체 완료!",https://blog.naver.com/rls0613/223324800050,비스포크키친핏 냉장고를 넣을 예정이라 이왕이면 냉장고장도 함께 할 수 있는 곳으로 ...,피프티 : 뷰티 인플루언서 블로그,blog.naver.com/rls0613,20240121
2,달항아리 오브제 냉장고 자석 마그네틱 조명 라디룸 무선 무드등,https://blog.naver.com/yseun89/223335360309,냉장고 자석 활용 가능한 부착형 마그네틱 조명이라는 거! 작년에 다이소에서 달항아리...,순간을 기록하는 공간.zip✨,blog.naver.com/yseun89,20240127
3,냉장고의 등짝을 보았다.,https://blog.naver.com/angelkittie/223334475815,"얌전하게 식혜를 통에 담아, 창가에 잠깐 놓아 (바깥이 섭씨로 영하 23도 25도 ...",아로리네,blog.naver.com/angelkittie,20240126
4,광주업소용주방기구 냉장고 가온주방에서 준비하기,https://blog.naver.com/cr_k_/223329919223,"이번에 사는 테이블, 의자, 냉장고, 작업대 등은 보다 신경을 많이 써서 골라야겠다...",짜리의 일상다반사 ♪,blog.naver.com/cr_k_,20240122


## 카페 검색 결과

In [44]:
def getresult(client_id,client_secret,query,display=100,start=1,sort='sim'):
    encText = urllib.parse.quote(query)
    url = "https://openapi.naver.com/v1/search/cafearticle?query=" + encText + \
    "&display=" + str(display) + "&start=" + str(start) + "&sort=" + sort

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        response_json = json.loads(response_body)
    else:
        print("Error Code:" + rescode)
    print(f"게시글 수 : {response_json['total']}")
    return pd.DataFrame(response_json['items']),response_json['total']

In [45]:
query = '냉장고'
display=100
start=1
sort='sim'

cafe_result,cafe_total= getresult(client_id,client_secret,query,display,start,sort)

cafe_result['title'] = cafe_result['title'].apply(lambda x: x.replace('&quot;','').replace('&apos;','').replace('<b>','').replace('</b>','').strip())
cafe_result['description'] = cafe_result['description'].apply(lambda x: x.replace('<b>','').replace('</b>','').replace('&quot;','').replace('&apos;','').strip())

cafe_result.head(5)

게시글 수 : 7381823


,title,link,description,cafename,cafeurl
0,냉장고 2대 위치 참견 부탁드립니다!,http://cafe.naver.com/overseer/1057472,33평 아파트에 키친핏 냉장고 4도어 + 김치냉장고 4도어가 들어갈 예정입니다. 주...,셀프 인테리어 My Home (혼자하는 집수...,https://cafe.naver.com/overseer
1,삼성냉장고~선택장애~의견들어요,http://cafe.naver.com/mokpomam/2181740,1.번 2번ㆍ 엘지15년 썼는데 곧 냉동실이 문제가 발생해서 바꿔야해요ㆍ 김냉 가능...,목포맘들의 수다방!!!,https://cafe.naver.com/mokpomam
2,엘지양문형냉장고 20% 특가중이네요!,http://cafe.naver.com/hotdealcommunity/2801764,엘지양문형냉장고 20% 특가중이네요! 엘지양문형냉장고 알아보시나요? 금액은 어느정도...,쇼핑매니아 [최저가 구매대행 카페],https://cafe.naver.com/hotdealcommunity
3,냉장고탈취제 추천받았는데 이게 맞나요?,http://cafe.naver.com/miznett/4052514,김장하고나서 김치냉장고가 갑자기 고장난 통에 김치를 냉장고에 넣어뒀더니 냄새가 장난...,미즈넷-임신 육아 임산부 교육 핫딜 아...,https://cafe.naver.com/miznett
4,냉장고 청소의 시작 ㅋ,http://cafe.naver.com/moymoy/894624,솔직히 냉장고 안은 수시로 하는거 같은데.. 그러니깐 겉면.. 특히 천장부분은 자주...,"★마더스 카페★ 책공구,교육,육아,코...",https://cafe.naver.com/moymoy
